In [3]:
import pandas as pd

pd.options.display.max_columns = 25
pd.options.display.max_rows = 48

df_json = pd.read_json('SNC.json').data.cards

df = pd.DataFrame(df_json)


df.drop(columns=['artist', 'availability', 'boosterTypes', 
                 'borderColor', 'finishes', 'foreignData', 
                 'frameVersion', 'hasFoil', 'hasNonFoil', 'edhrecRank',
                 'identifiers', 'language', 'layout', 'legalities',
                 'number', 'printings', 'purchaseUrls',
                 'rulings', 'watermark', 'securityStamp',
                 'variations', 'leadershipSkills', 'frameEffects',
                 'isStorySpotlight', 'isReprint', 'isFullArt', 
                 'isStarter', 'promoTypes', 'isPromo'], inplace=True)


#Finding duplicates and their index based on the name
nondupeindex = df['name'].drop_duplicates().index

#Create a new dataframe with the duplicates removed
rdf = df[df.index.isin(nondupeindex)]


#Create the card_stats table
card_stats = rdf[['colorIdentity', 'colors', 'convertedManaCost', 'manaCost', 
                  'manaValue', 'power', 'toughness', 'loyalty', 'rarity', 'setCode']].copy()

card_stats.insert(0, 'card_stats_id', range(1, len(card_stats) + 1))
#card_stats.insert(1, 'color_id', range(1, len(card_stats) + 1))

card_stats.insert(1, 'color_id', range(1035, len(card_stats) + 1035))

#color_class.rename(columns={'color_id': 'color_class_id', 'colorIdentity': 'color_identity'},inplace=True)
card_stats.rename(columns={'colorIdentity': 'color_identity', 'convertedManaCost': 'coverted_mana_cost', 
                           'manaCost': 'mana_cost', 'setCode': 'set_code'}, inplace=True)

#Create the card_text table
card_text = rdf[['name', 'flavorText', 'text', 'keywords', 'type', 'supertypes', 'types', 'subtypes']].copy()
card_text.rename(columns={'flavorText': 'flavor_text'}, inplace=True)

card_text.insert(0, 'card_text_id', range(10979, len(card_text) + 10979)) 
card_text.insert(1, 'card_typing_id', range(8707, len(card_text) + 8707)) 

#Creating the keyword table
keywords = card_text[['card_text_id', 'keywords']].explode('keywords')

#Creating the types table
types = card_text[['card_typing_id', 'types']].explode('types')

#Creating the subtypes table
subtypes = card_text[['card_typing_id', 'subtypes']].explode('subtypes')


mcf = card_stats[['color_id', 'mana_cost']]

def set_colors(x, c): 
    return x.mana_cost.astype(str).str.count(c)

#Creating the mana_cost table
mana_cost = mcf.assign(colorless='placeholder',
               white=lambda x: set_colors(x, 'W'), 
               blue=lambda x: set_colors(x, 'U'),
               black=lambda x: set_colors(x, 'B'),
               red=lambda x: set_colors(x, 'R'),
               green=lambda x: set_colors(x, 'G'))

mana_cost.rename(columns={'color_id': 'mana_cost_id'},inplace=True)

def make_colorless(x):
    res = []
    for item in x:
        if item[1] == 'X':
            res.append('X')
        elif item[1].isnumeric():
            res.append(item[1])
        else: 
            res.append('0')
    return res

mana_cost_list = card_stats['mana_cost'].fillna('{0}').tolist()

colorless_list = pd.Series(make_colorless(mana_cost_list))

mana_cost.reset_index(inplace=True)
mana_cost['colorless'] = colorless_list

mana_cost.drop(columns=['index'], inplace=True)

def is_color_present(x, c):
    return x.astype(str).str.count(c)


#Creating the color_class table
color_class = card_stats[['color_id', 'color_identity']].copy()
color_class.rename(columns={'color_id': 'color_class_id'},inplace=True)


color_class = color_class.assign(white=lambda x: is_color_present(x.color_identity, 'W'),
                                blue=lambda x: is_color_present(x.color_identity, 'U'),
                                black=lambda x: is_color_present(x.color_identity, 'B'),
                                red=lambda x: is_color_present(x.color_identity, 'R'),
                                green=lambda x: is_color_present(x.color_identity, 'G'))

#Creating the colors table
colors = card_stats[['color_id', 'colors']].copy()
colors.rename(columns={'color_id': 'colors_id'},inplace=True)

colors = colors.assign(white=lambda x: is_color_present(x.colors, 'W'),
                        blue=lambda x: is_color_present(x.colors, 'U'),
                        black=lambda x: is_color_present(x.colors, 'B'),
                        red=lambda x: is_color_present(x.colors, 'R'),
                        green=lambda x: is_color_present(x.colors, 'G'))


,card_text_id,card_typing_id,name,flavor_text,text,keywords,type,supertypes,types,subtypes
0,10979,8707,Angelic Observer,Still and solemn as the statues of her kind th...,This spell costs {1} less to cast for each Cit...,[Flying],Creature — Angel Advisor,[],[Creature],"[Angel, Advisor]"
1,10980,8708,Backup Agent,"""My sources say the Beamtown Bullies were spot...","When Backup Agent enters the battlefield, put ...",NaN,Creature — Human Citizen,[],[Creature],"[Human, Citizen]"
2,10981,8709,Ballroom Brawlers,Don't try to cut in. They'll cut back.,"Whenever Ballroom Brawlers attacks, Ballroom B...",NaN,Creature — Human Warrior,[],[Creature],"[Human, Warrior]"
3,10982,8710,Boon of Safety,"""Saving a stranger's life is an excellent recr...",Put a shield counter on target creature. (If i...,[Scry],Instant,[],[Instant],[]
4,10983,8711,Brokers Initiate,"Regarding the relative might of pen and sword,...",{4}{G/U}: Brokers Initiate has base power and ...,NaN,Creature — Cat Citizen,[],[Creature],"[Cat, Citizen]"
...,...,...,...,...,...,...,...,...,...,...
261,11240,8968,Plains,NaN,({T}: Add {W}.),NaN,Basic Land — Plains,[Basic],[Land],[Plains]
263,11241,8969,Island,NaN,({T}: Add {U}.),NaN,Basic Land — Island,[Basic],[Land],[Island]
265,11242,8970,Swamp,NaN,({T}: Add {B}.),NaN,Basic Land — Swamp,[Basic],[Land],[Swamp]
267,11243,8971,Mountain,NaN,({T}: Add {R}.),NaN,Basic Land — Mountain,[Basic],[Land],[Mountain]
